In [1]:
import requests
from bs4 import BeautifulSoup


In [2]:
URL = "https://gymnasium.farama.org/environments/atari/"
response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")


In [3]:
games = {}

table = soup.find(id="flavors").find("table")
rows = table.find_all("tr")
for row in rows:
    cells = row.find_all("td")
    if len(cells) == 0:
        continue

    data = {}
    name = cells[0].text
    modes = cells[1].text.split(", ")
    default_mode = cells[2].text
    difficulties = cells[3].text.split(", ")
    default_difficulty = cells[4].text

    games[name.lower()] = {
        "name": name,
        "modes": modes,
        "default_mode": default_mode,
        "difficulties": difficulties,
        "default_difficulty": default_difficulty,
    }


In [4]:
print(games)


{'adventure': {'name': 'Adventure', 'modes': ['[0', '1', '2]'], 'default_mode': '0', 'difficulties': ['[0', '1', '2', '3]'], 'default_difficulty': '0'}, 'airraid': {'name': 'AirRaid', 'modes': ['[1', '…', '8]'], 'default_mode': '1', 'difficulties': ['[0]'], 'default_difficulty': '0'}, 'alien': {'name': 'Alien', 'modes': ['[0', '1', '2', '3]'], 'default_mode': '0', 'difficulties': ['[0', '1', '2', '3]'], 'default_difficulty': '0'}, 'amidar': {'name': 'Amidar', 'modes': ['[0]'], 'default_mode': '0', 'difficulties': ['[0', '3]'], 'default_difficulty': '0'}, 'assault': {'name': 'Assault', 'modes': ['[0]'], 'default_mode': '0', 'difficulties': ['[0]'], 'default_difficulty': '0'}, 'asterix': {'name': 'Asterix', 'modes': ['[0]'], 'default_mode': '0', 'difficulties': ['[0]'], 'default_difficulty': '0'}, 'asteroids': {'name': 'Asteroids', 'modes': ['[0', '…', '31', '128]'], 'default_mode': '0', 'difficulties': ['[0', '3]'], 'default_difficulty': '0'}, 'atlantis': {'name': 'Atlantis', 'modes': [

In [5]:
URL = "https://gymnasium.farama.org/environments/atari/complete_list/"
response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")


In [6]:
section = soup.find(id="complete-list-atari").find(class_="env-grid")
anchors = section.find_all("a")

for anchor in anchors:
    title = anchor.find(class_="cell__title").text.lower().strip()
    name = title.replace(" ", "")
    slug = title.replace(" ", "-")
    games[name]["url"] = URL + anchor["href"]
    games[name]["slug"] = slug


In [7]:
print(games)


{'adventure': {'name': 'Adventure', 'modes': ['[0', '1', '2]'], 'default_mode': '0', 'difficulties': ['[0', '1', '2', '3]'], 'default_difficulty': '0', 'url': 'https://gymnasium.farama.org/environments/atari/complete_list/../adventure', 'slug': 'adventure'}, 'airraid': {'name': 'AirRaid', 'modes': ['[1', '…', '8]'], 'default_mode': '1', 'difficulties': ['[0]'], 'default_difficulty': '0', 'url': 'https://gymnasium.farama.org/environments/atari/complete_list/../air_raid', 'slug': 'air-raid'}, 'alien': {'name': 'Alien', 'modes': ['[0', '1', '2', '3]'], 'default_mode': '0', 'difficulties': ['[0', '1', '2', '3]'], 'default_difficulty': '0', 'url': 'https://gymnasium.farama.org/environments/atari/complete_list/../alien', 'slug': 'alien'}, 'amidar': {'name': 'Amidar', 'modes': ['[0]'], 'default_mode': '0', 'difficulties': ['[0', '3]'], 'default_difficulty': '0', 'url': 'https://gymnasium.farama.org/environments/atari/complete_list/../amidar', 'slug': 'amidar'}, 'assault': {'name': 'Assault', 

In [9]:
for game in games.values():
    response = requests.get(game["url"])
    soup = BeautifulSoup(response.text, "html.parser")

    game["soup"] = soup


In [12]:
for game in games.values():
    txt = game["soup"].find(class_="docutils literal notranslate").text.strip()
    game["env"] = txt.split('"')[1]


In [24]:
for game in games.values():
    game["description"] = game["soup"].find(id="description").find("p").text.strip()


In [26]:
print(games["asterix"]["description"])


You are Asterix and can move horizontally (continuously) and vertically (discretely). Objects move horizontally across the screen: lyres and other (more useful) objects. Your goal is to guideAsterix in such a way as to avoid lyres and collect as many other objects as possible. You score points by collecting objects and lose a life whenever you collect a lyre. You have three lives available at the beginning. If you score sufficiently many points, you will be awarded additional points.


In [34]:
from copy import deepcopy
from json import dumps

new_games = deepcopy(games)

for game in new_games.values():
    del game["soup"]

    modes = []
    for mode in game["modes"]:
        modes.append(mode.lower().replace("[", "").replace("]", "").strip())

    game["modes"] = modes

    difficulties = []
    for difficulty in game["difficulties"]:
        difficulties.append(
            difficulty.lower().replace("[", "").replace("]", "").strip()
        )

    game["difficulties"] = difficulties


with open("games.json", "w") as f:
    f.write(dumps(new_games))
